# Fundamental Factors
#### Good Indicators
###### FCF to EV +
###### CapEx Vol +
###### Debt to Total Assets +
###### *capex and debt to assets highly correlated

#### Bad Indicators
###### Sales Size

In [ ]:
import numpy as np
import math
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import QTradableStocksUS

from quantopian.pipeline.factors import Latest
from quantopian.pipeline.data import morningstar,Fundamentals
from quantopian.pipeline.factors import CustomFactor, SimpleMovingAverage, AverageDollarVolume, SimpleBeta,Returns, RSI
from quantopian.pipeline.classifiers.morningstar import Sector
from quantopian.pipeline.classifiers.fundamentals import Sector
from quantopian.pipeline.data.zacks import EarningsSurprises
from quantopian.pipeline.data import factset
from quantopian.pipeline.data.psychsignal import stocktwits

#Python imports
import math
import talib
import numpy as np
import pandas as pd
import pyfolio as pf
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import linear_model, decomposition, ensemble, preprocessing, isotonic, metrics
from scipy.stats.mstats import winsorize
from zipline.utils.numpy_utils import (
    repeat_first_axis,
    repeat_last_axis,
)
from scipy.stats.mstats import gmean

bs = morningstar.balance_sheet
cfs = morningstar.cash_flow_statement
is_ = morningstar.income_statement
or_ = morningstar.operation_ratios
er = morningstar.earnings_report
v = morningstar.valuation
vr = morningstar.valuation_ratios

# FCF to EV

##### Free cash flow (FCF) is a measure of how much cash a company has on hand after all expenses are extracted. High FCF indicates that larger amounts of cash are available to the company for reinvestment. By dividing by a company’s enterprise value (EV), we can compute a ratio that shows how cash is generated per unit of the value of a company. In this implementation, we can test the idea that companies with a relatively higher ratio of FCF/EV are likely to outperform companies with relatively lower levels of FCF/EV.

In [ ]:
def make_pipeline():
    ZSCORE_FILTER = 3 # Maximum number of standard deviations to include before counting as outliers
    ZERO_FILTER = 0.001 # Minimum weight we allow before dropping security

   # Setting up the variables
    alpha_factor = factset.Fundamentals.free_cf_fcfe_qf.latest / \
                   factset.Fundamentals.entrpr_val_qf.latest
 
    # Standardized logic for each input factor after this point
    alpha_w = alpha_factor.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=QTradableStocksUS() & alpha_factor.isfinite())
 
    alpha_z = alpha_w.zscore()
    alpha_weight = alpha_z / 100.0
 
    outlier_filter = alpha_z.abs() < 3
    zero_filter = alpha_weight.abs() > 0.001
 
    universe = QTradableStocksUS() & \
               outlier_filter & \
               zero_filter
    
    sector = Sector()
 
    pipe = Pipeline(
        columns={
            'alpha_weight': alpha_weight,
            'sector': sector,
        },
        screen=universe)
        
    return pipe
    

In [ ]:
result = run_pipeline(make_pipeline(), start_date='2015-01-01',end_date='2016-01-01')
result.head()

In [ ]:
assets = result.index.levels[1].unique()
len(assets)

In [ ]:
pricing_data = get_pricing(assets,start_date='2014-06-01',end_date='2016-06-01', fields='open_price')

In [ ]:
import alphalens as al
from alphalens.utils import get_clean_factor_and_forward_returns
from alphalens.tears import create_full_tear_sheet

sector_labels, sector_labels[-1] = dict(Sector.SECTOR_NAMES), "Unknown"

factor_data = get_clean_factor_and_forward_returns(
    result['alpha_weight'], # How to analyze a specific pipeline column with Alphalens
    pricing_data,
    quantiles = 2,
    periods=(21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)
create_full_tear_sheet(factor_data=factor_data, by_group=True)

# Debt to Total Assets

##### The ratio of a company’s debt to its total assets is a measure of the amount of leverage taken on by a company. Higher values (above 1) indicate that a company has more liabilities than assets (aka you owe more than you own), while lower values (below 1) indicate that a company has more equity than debt. In this template, we’ve taken advantage of this measure to test the idea that companies with relatively lower levels of Debt to Total Assets are likely to outperform companies with relatively higher levels of Debt to Total Assets. 

In [ ]:
def make_pipeline():
    ZSCORE_FILTER = 3 # Maximum number of standard deviations to include before counting as outliers
    ZERO_FILTER = 0.001 # Minimum weight we allow before dropping security

   # Setting up the variables
    alpha_factor = -factset.Fundamentals.debt.latest / \
                    factset.Fundamentals.assets.latest
    
    # Standardized logic for each input factor after this point
    alpha_w = alpha_factor.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=QTradableStocksUS() & alpha_factor.isfinite())
    
    alpha_z = alpha_w.zscore()
    alpha_weight = alpha_z / 100.0
    
    outlier_filter = alpha_z.abs() < ZSCORE_FILTER
    zero_filter = alpha_weight.abs() > ZERO_FILTER
 
    universe = QTradableStocksUS() & \
               outlier_filter & \
               zero_filter
    
    sector = Sector()
 
    pipe = Pipeline(
        columns={
            'alpha_weight': alpha_weight,
            'sector': sector,
        },
        screen=universe
    )
    return pipe
    

In [ ]:
result = run_pipeline(make_pipeline(), start_date='2015-01-01',end_date='2016-01-01')
result.head()

In [ ]:
assets = result.index.levels[1].unique()
len(assets)

In [ ]:
pricing_data = get_pricing(assets,start_date='2014-05-01',end_date='2016-07-01', fields='open_price')

In [ ]:
import alphalens as al
from alphalens.utils import get_clean_factor_and_forward_returns
from alphalens.tears import create_full_tear_sheet

sector_labels, sector_labels[-1] = dict(Sector.SECTOR_NAMES), "Unknown"

factor_data = get_clean_factor_and_forward_returns(
    result['alpha_weight'], # How to analyze a specific pipeline column with Alphalens
    pricing_data,
    quantiles = 2,
    periods=(21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)
create_full_tear_sheet(factor_data=factor_data, by_group=True)

# CapEx Vol

##### Cash flow volatility is a fairly well studied metric that is often considered a proxy for uncertainty at a firm level. In this template algorithm we've extended that idea to see if firms with relatively more volatile capital expenditures (e.g. spending on things like new buildings, plants, equipment, etc) are also more unpredictable and, by extension, riskier and more likely to underperform firms with lower relative capex volatility. For a bit more academic detail take a look at this SSRN paper.

In [ ]:
class TEM(CustomFactor):
    """
    TEM = standard deviation of past 6 quarters' reports
    """
    window_length = 390
    def compute(self, today, assets, out, asof_date, capex, total_assets):
        values = capex/total_assets
        for column_ix in range(asof_date.shape[1]):
            _, unique_indices = np.unique(asof_date[:, column_ix], return_index=True)
            quarterly_values = values[unique_indices, column_ix]
            if len(quarterly_values) < 6:
                quarterly_values = np.hstack([
                    np.repeat([np.nan], 6 - len(quarterly_values)),
                    quarterly_values,
                ])
            out[column_ix] = np.std(quarterly_values[-6:])

In [ ]:
def make_pipeline():
    ZSCORE_FILTER = 3 # Maximum number of standard deviations to include before counting as outliers
    ZERO_FILTER = 0.001 # Minimum weight we allow before dropping security
    # Setting up the variables
    capex_vol = TEM(
        inputs=[factset.Fundamentals.capex_qf_asof_date,
                factset.Fundamentals.capex_qf,
                factset.Fundamentals.assets],
        mask=QTradableStocksUS()
    )
    
    alpha_factor = -capex_vol
    
    # Standardized logic for each input factor after this point
    alpha_w = alpha_factor.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=alpha_factor.isfinite())
    
    alpha_z = alpha_w.zscore()
    alpha_weight = alpha_z / 100.0
    
    outlier_filter = alpha_z.abs() < ZSCORE_FILTER
    zero_filter = alpha_weight.abs() > ZERO_FILTER
 
    universe = QTradableStocksUS() & \
               outlier_filter & \
               zero_filter
    
    sector = Sector()
 
    pipe = Pipeline(
        columns={
            'alpha_weight': alpha_weight,
            'sector': sector,
        },
        screen=universe
    )
    return pipe

In [ ]:
result = run_pipeline(make_pipeline(), start_date='2015-01-01',end_date='2016-01-01')
result.head()

In [ ]:
assets = result.index.levels[1].unique()
len(assets)

In [ ]:
pricing_data = get_pricing(assets, start_date = '2014-05-01', end_date = '2016-07-01', fields = 'open_price')

In [ ]:
import alphalens as al
from alphalens.utils import get_clean_factor_and_forward_returns
from alphalens.tears import create_full_tear_sheet

sector_labels, sector_labels[-1] = dict(Sector.SECTOR_NAMES), "Unknown"

factor_data = get_clean_factor_and_forward_returns(
    result['alpha_weight'],
    pricing_data,
    quantiles = 2,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data=factor_data, by_group=True)

###### Going Sector Neutral

In [ ]:
factor_data = get_clean_factor_and_forward_returns(
    result['alpha_weight'],
    pricing_data,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
    binning_by_group=True,
)

create_full_tear_sheet(factor_data=factor_data, by_group=True,group_neutral=True)

# Sales Size

##### This is a valuation metric. We expect companies that generate more revenue per dollar of market cap to generate outsized returns compared to a company of comparable size that generates less revenue. Our factor is calculated as the sale of revenue from goods and services over the last twelve months divided by market cap. We would naturally expect larger companies to generate more raw sales revenue than smaller companies, so we try to remove that from the equation and get at pure value.

In [ ]:
def make_pipeline():
    OUTLIER_THRESHOLD = 3 # Maximum zscore that is not an outlier
    ZERO_THRESHOLD = 0.1 # Minimum zscore we allow before dropping security
    # Setting up the variables
    revenue_ltm = factset.Fundamentals.sales_ltm.latest/\
                  factset.Fundamentals.mkt_val_public.latest
    alpha_factor = revenue_ltm.log()
    
    # Standardized logic for each input factor after this point
    alpha_w = alpha_factor.winsorize(
        min_percentile=0.02,
        max_percentile=0.98,
        mask=QTradableStocksUS() & (revenue_ltm > 0) & alpha_factor.isfinite()
    )
    alpha_z = alpha_w.zscore()
    
    outlier_filter = alpha_z.abs() < OUTLIER_THRESHOLD
    non_zero_filter = alpha_z.abs() > ZERO_THRESHOLD
    
    universe = QTradableStocksUS() & \
               outlier_filter & \
               non_zero_filter
    
    sector = Sector()
 
    pipe = Pipeline(
        columns={
            'alpha_z': alpha_z,
            'sector': sector,
        },
        screen=universe
    )
    return pipe


In [ ]:
result = run_pipeline(make_pipeline(), start_date = '2015-01-01', end_date = '2016-01-01')
result.head()

In [ ]:
assets = result.index.levels[1].unique()
len(assets)

In [ ]:
pricing_data = get_pricing(assets, start_date = '2014-06-01', end_date = '2016-07-01', fields = 'open_price')

In [ ]:
import alphalens as al
from alphalens.utils import get_clean_factor_and_forward_returns
from alphalens.tears import create_full_tear_sheet

sector_labels, sector_labels[-1] = dict(Sector.SECTOR_NAMES), "Unknown"

factor_data = get_clean_factor_and_forward_returns(
    result['alpha_z'],
    pricing_data, 
    quantiles = 2, 
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data=factor_data, by_group=True)

In [ ]:
from alphalens.performance import mean_information_coefficient
mean_information_coefficient(factor_data).plot(title="IC Decay");

# Combination

In [ ]:
def make_pipeline():
    ZSCORE_FILTER = 3 # Maximum number of standard deviations to include before counting as outliers
    ZERO_FILTER = 0.001 # Minimum weight we allow before dropping security
    
    # ALPHA FACTOR 1
    alpha_factor1 = factset.Fundamentals.free_cf_fcfe_qf.latest / \
                   factset.Fundamentals.entrpr_val_qf.latest
 
    # Standardized logic for each input factor after this point
    alpha_w1 = alpha_factor1.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=QTradableStocksUS() & alpha_factor1.isfinite())
 
    alpha_z1 = alpha_w1.zscore()
    alpha_weight1 = alpha_z1 / 100.0
 
    outlier_filter1 = alpha_z1.abs() < 3
    zero_filter1 = alpha_weight1.abs() > 0.001
 
    universe1 = QTradableStocksUS() & \
               outlier_filter1 & \
               zero_filter1
    
    # ALPHA FACTOR 2
    alpha_factor2 = -factset.Fundamentals.debt.latest / \
                    factset.Fundamentals.assets.latest
    
    # Standardized logic for each input factor after this point
    alpha_w2 = alpha_factor2.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=QTradableStocksUS() & alpha_factor2.isfinite())
    
    alpha_z2 = alpha_w2.zscore()
    alpha_weight2 = alpha_z2 / 100.0
    
    outlier_filter2 = alpha_z2.abs() < ZSCORE_FILTER
    zero_filter2 = alpha_weight2.abs() > ZERO_FILTER
 
    universe2 = QTradableStocksUS() & \
               outlier_filter2 & \
               zero_filter2
 
    # ALPHA FACTOR 3
    capex_vol = TEM(
        inputs=[factset.Fundamentals.capex_qf_asof_date,
                factset.Fundamentals.capex_qf,
                factset.Fundamentals.assets],
        mask=QTradableStocksUS()
    )
    
    alpha_factor3 = -capex_vol
    
    # Standardized logic for each input factor after this point
    alpha_w3 = alpha_factor3.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=alpha_factor3.isfinite())
    
    alpha_z3 = alpha_w3.zscore()
    alpha_weight3 = alpha_z3 / 100.0
    
    outlier_filter3 = alpha_z3.abs() < ZSCORE_FILTER
    zero_filter3 = alpha_weight3.abs() > ZERO_FILTER
 
    universe3 = QTradableStocksUS() & \
               outlier_filter3 & \
               zero_filter3
    
#    universe = (QTradableStocksUS() 
#               & testing_factor1.notnull()
 #               & testing_factor2.notnull()
  #              & testing_factor3.notnull())

    OUTLIER_THRESHOLD = 3 # Maximum zscore that is not an outlier
    ZERO_THRESHOLD = 0.1 # Minimum zscore we allow before dropping security
    # ALPHA FACTOR 4
    revenue_ltm = factset.Fundamentals.sales_ltm.latest/\
                  factset.Fundamentals.mkt_val_public.latest
    alpha_factor4 = revenue_ltm.log()
    
    # Standardized logic for each input factor after this point
    alpha_w4 = alpha_factor4.winsorize(
        min_percentile=0.02,
        max_percentile=0.98,
        mask=QTradableStocksUS() & (revenue_ltm > 0) & alpha_factor4.isfinite()
    )
    alpha_z4 = alpha_w4.zscore()
    alpha_weight4 = alpha_z4 /100
    
    outlier_filter4 = alpha_z4.abs() < OUTLIER_THRESHOLD
    non_zero_filter4 = alpha_z4.abs() > ZERO_THRESHOLD
    
    universe4 = QTradableStocksUS() & \
               outlier_filter4 & \
               non_zero_filter4
    
    universe = universe1 & universe2 & universe3 & universe4
    
    alpha_weight = alpha_weight1 + alpha_weight2 + alpha_weight3 + alpha_weight4
    
    testing_quantiles = alpha_weight.quantiles(2)
    
    pipe = Pipeline(
        columns={
            'alpha_weight': alpha_weight,
            'shorts':testing_quantiles.eq(0),
            'longs':testing_quantiles.eq(1),
        },
        screen=universe
    )
    return pipe
    

In [ ]:
result = run_pipeline(make_pipeline(), start_date = '2015-01-01', end_date = '2016-01-01')
result.head()

In [ ]:
assets = result.index.levels[1].unique()
len(assets)

In [ ]:
pricing_data = get_pricing(assets, start_date = '2014-05-01', end_date = '2016-08-01', fields = 'open_price')

In [ ]:
import alphalens as al
from alphalens.utils import get_clean_factor_and_forward_returns
from alphalens.tears import create_full_tear_sheet

factor_data = get_clean_factor_and_forward_returns(
    result['alpha_weight'],
    pricing_data,
    quantiles = 2,
    periods = (21,63,126))

create_full_tear_sheet(factor_data = factor_data)

# Correlation of Factors

In [ ]:
def make_pipeline():
    ZSCORE_FILTER = 3 # Maximum number of standard deviations to include before counting as outliers
    ZERO_FILTER = 0.001 # Minimum weight we allow before dropping security
    
    # ALPHA FACTOR 1
    alpha_factor1 = factset.Fundamentals.free_cf_fcfe_qf.latest / \
                   factset.Fundamentals.entrpr_val_qf.latest
 
    # Standardized logic for each input factor after this point
    alpha_w1 = alpha_factor1.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=QTradableStocksUS() & alpha_factor1.isfinite())
 
    alpha_z1 = alpha_w1.zscore()
    alpha_weight1 = alpha_z1 / 100.0
 
    outlier_filter1 = alpha_z1.abs() < 3
    zero_filter1 = alpha_weight1.abs() > 0.001
 
    universe1 = QTradableStocksUS() & \
               outlier_filter1 & \
               zero_filter1
    
    # ALPHA FACTOR 2
    alpha_factor2 = -factset.Fundamentals.debt.latest / \
                    factset.Fundamentals.assets.latest
    
    # Standardized logic for each input factor after this point
    alpha_w2 = alpha_factor2.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=QTradableStocksUS() & alpha_factor2.isfinite())
    
    alpha_z2 = alpha_w2.zscore()
    alpha_weight2 = alpha_z2 / 100.0
    
    outlier_filter2 = alpha_z2.abs() < ZSCORE_FILTER
    zero_filter2 = alpha_weight2.abs() > ZERO_FILTER
 
    universe2 = QTradableStocksUS() & \
               outlier_filter2 & \
               zero_filter2
 
    # ALPHA FACTOR 3
    capex_vol = TEM(
        inputs=[factset.Fundamentals.capex_qf_asof_date,
                factset.Fundamentals.capex_qf,
                factset.Fundamentals.assets],
        mask=QTradableStocksUS()
    )
    
    alpha_factor3 = -capex_vol
    
    # Standardized logic for each input factor after this point
    alpha_w3 = alpha_factor3.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=alpha_factor3.isfinite())
    
    alpha_z3 = alpha_w3.zscore()
    alpha_weight3 = alpha_z3 / 100.0
    
    outlier_filter3 = alpha_z3.abs() < ZSCORE_FILTER
    zero_filter3 = alpha_weight3.abs() > ZERO_FILTER
 
    universe3 = QTradableStocksUS() & \
               outlier_filter3 & \
               zero_filter3
    
#    universe = (QTradableStocksUS() 
#               & testing_factor1.notnull()
 #               & testing_factor2.notnull()
  #              & testing_factor3.notnull())

    OUTLIER_THRESHOLD = 3 # Maximum zscore that is not an outlier
    ZERO_THRESHOLD = 0.1 # Minimum zscore we allow before dropping security
    # ALPHA FACTOR 4
    revenue_ltm = factset.Fundamentals.sales_ltm.latest/\
                  factset.Fundamentals.mkt_val_public.latest
    alpha_factor4 = revenue_ltm.log()
    
    # Standardized logic for each input factor after this point
    alpha_w4 = alpha_factor4.winsorize(
        min_percentile=0.02,
        max_percentile=0.98,
        mask=QTradableStocksUS() & (revenue_ltm > 0) & alpha_factor4.isfinite()
    )
    alpha_z4 = alpha_w4.zscore()
    alpha_weight4 = alpha_z4 /100
    
    outlier_filter4 = alpha_z4.abs() < OUTLIER_THRESHOLD
    non_zero_filter4 = alpha_z4.abs() > ZERO_THRESHOLD
    
    universe4 = QTradableStocksUS() & \
               outlier_filter4 & \
               non_zero_filter4
    
    universe = universe1 & universe2 & universe3 & universe4
    
    alpha_weight = alpha_weight1 + alpha_weight2 + alpha_weight3 + alpha_weight4
    
    testing_quantiles = alpha_weight.quantiles(2)
    
    pipe = Pipeline(
        columns={
            'alpha_weight': alpha_weight,
            'fcf_to_ev': alpha_weight1,
            'debt_to_assets': alpha_weight2,
            'capex_vol':alpha_weight3,
            'sales_size': alpha_weight4,
            'sector': Sector(mask=universe),
        },
        screen=universe
    )
    return pipe
    

In [ ]:
result = run_pipeline(make_pipeline(), start_date = '2015-01-01', end_date = '2016-01-01')
result = result.dropna()
result.head()

In [ ]:
assets = result.index.levels[1].unique()
len(assets)

In [ ]:
pricing_data = get_pricing(assets, start_date = '2014-06-01', end_date = '2016-07-01', fields='open_price')

In [ ]:
import alphalens as al

sector_labels = dict(Sector.SECTOR_NAMES)
sector_labels[-1] = "Unknown" # no dataset is perfect, better handle the unexpected

In [ ]:
factor1_data = al.utils.get_clean_factor_and_forward_returns(
    result["fcf_to_ev"],
    pricing_data,
    groupby=result["sector"],
    quantiles=5,
    periods=(21, 63, 126)
)

factor2_data = al.utils.get_clean_factor_and_forward_returns(
    result["debt_to_assets"],
    pricing_data,
    groupby=result["sector"],
    quantiles=5,
    periods=(21, 63, 126)
)

factor3_data = al.utils.get_clean_factor_and_forward_returns(
    result["capex_vol"],
    pricing_data,
    groupby=result["sector"],
    quantiles=5,
    periods=(21, 63, 126)
)

factor4_data = al.utils.get_clean_factor_and_forward_returns(
    result["sales_size"],
    pricing_data,
    groupby=result["sector"],
    quantiles=5,
    periods=(21, 63, 126)
)

In [ ]:
factor1_returns, factor1_positions, factor1_benchmark = \
    al.performance.create_pyfolio_input(factor1_data,
                                        period='63D',
                                        capital=1000000,
                                        long_short=True,
                                        group_neutral=False,
                                        equal_weight=True,
                                        quantiles=[1,5],
                                        groups=None,
                                        benchmark_period='21D')

factor2_returns, factor2_positions, factor2_benchmark = \
    al.performance.create_pyfolio_input(factor2_data,
                                        period='63D',
                                        capital=1000000,
                                        long_short=True,
                                        group_neutral=False,
                                        equal_weight=True,
                                        quantiles=[1,5],
                                        groups=None,
                                        benchmark_period='21D')

factor3_returns, factor3_positions, factor3_benchmark = \
    al.performance.create_pyfolio_input(factor3_data,
                                        period='63D',
                                        capital=1000000,
                                        long_short=True,
                                        group_neutral=False,
                                        equal_weight=True,
                                        quantiles=[1,5],
                                        groups=None,
                                        benchmark_period='21D')

factor4_returns, factor4_positions, factor4_benchmark = \
    al.performance.create_pyfolio_input(factor4_data,
                                        period='63D',
                                        capital=1000000,
                                        long_short=True,
                                        group_neutral=False,
                                        equal_weight=True,
                                        quantiles=[1,5],
                                        groups=None,
                                        benchmark_period='21D')

In [ ]:
import matplotlib.pyplot as plt
factor1_returns.plot()
factor2_returns.plot()
factor3_returns.plot()
factor4_returns.plot()
plt.ylabel('Returns')
plt.legend(['Factor1', 'Factor2', 'Factor3','Factor4']);

##### Analysis of Correlations: 
###### FCF_to_Ev not correlated to debt_to_assets, somewhat correlated to CapexVol and Salessize
###### Debt_to_assets highly positvely correlated to CapexVol 
###### Debt to assets highly negatively correlated to sales size
###### CapexVol highly negatively correlated to Sales size

###### Keep FCF_to_EV, Sales_Size(negative correlated is that good?)
###### Decide between Debt to Assets or Capex Vol


